In [12]:
import torch
from dptb.nnops.nnapi import NNSK
from dptb.structure.structure import BaseStruct
from dptb.dataprocess.datareader import read_data
import matplotlib.pyplot as plt
import pickle as pickle
import numpy as np


if __name__ == '__main__':
    checkpointPath = "/root/dptb_exp/hBN/checkpoint/latest_nnsk.pth"
    train_data_path = "/root/dptb_exp/hBN/data/"
    train_data_prefix = "set"

    proj_atom_neles = {"N": 5, "B": 3}
    proj_anglr_m = {
      "N": [
        "2s",
        "2p",
        "s*"
      ],
      "B": [
        "2s",
        "2p",
        "s*"
      ]
    }
    nnsk = NNSK(checkpointPath, proj_anglr_m)
    band = pickle.load(open('./TBmodel/hBN/check/band_structure.pickle', 'rb'))

    filename = './TBmodel/hBN/check/hBN.vasp'
    st = BaseStruct(atom=filename, format='vasp',
                    cutoff=3.6, proj_atom_anglr_m=proj_anglr_m, proj_atom_neles=proj_atom_neles, onsitemode='uniform')

    snapase = st.struct
    # snapase = structase
    lat = snapase.cell.get_bravais_lattice()
    print(lat.description())
    # lat.plot_bz(show=True)
    special_kp = lat.get_special_points()
    # spmap['M'] = np.array([0.5,0.5,1])
    # kpath=snapase.cell.bandpath('XGX', npoints=120)
    kpath = snapase.cell.bandpath('GMKG', npoints=400)
    xlist, high_sym_kpoints, labels = kpath.get_linear_kpoint_axis()
    klist = kpath.kpts
    
    all_bonds, hamil_blocks, overlap_blocks = nnsk.get_HR(st)
    hkmat, skmat = nnsk.get_HK([klist[0]])
    #### here
    nnsk = NNSK(checkpointPath, proj_anglr_m)
    band = pickle.load(open('./TBmodel/hBN/check/band_structure.pickle', 'rb'))
    filename = './TBmodel/hBN/check/hBN.vasp'
    st = BaseStruct(atom=filename, format='vasp',
                    cutoff=2.6, proj_atom_anglr_m=proj_anglr_m, proj_atom_neles=proj_atom_neles, onsitemode='uniform')

    snapase = st.struct
    # snapase = structase
    lat = snapase.cell.get_bravais_lattice()
    print(lat.description())
    # lat.plot_bz(show=True)
    special_kp = lat.get_special_points()
    # spmap['M'] = np.array([0.5,0.5,1])
    # kpath=snapase.cell.bandpath('XGX', npoints=120)
    kpath = snapase.cell.bandpath('GMKG', npoints=400)
    xlist, high_sym_kpoints, labels = kpath.get_linear_kpoint_axis()
    klist = kpath.kpts
    all_bonds, hamil_blocks, overlap_blocks = nnsk.get_HR(st)
    hkmat_, skmat_ = nnsk.get_HK([klist[0]])

    plt.matshow(hkmat[0].detach())
    plt.show()
    plt.matshow(hkmat_[0].detach())
    plt.show()
    # for i in hamil_blocks:
    #   print(i)
    # eigks, EF = nnsk.get_eigenvalues(klist)

    # plt.figure(figsize=(5, 5), dpi=100)
    # for i in range(eigks.shape[1]):
    #     plt.plot(xlist, eigks[:, i] - np.min(eigks), 'r-', lw=1)

    # for ib in range(band['eigenvalues'].shape[1]):
    #     plt.plot(band['xcoords'], band['eigenvalues'][:, ib] - np.min(band['eigenvalues']), 'k--')
    # # plt.plot(band['xcoords'],band['eigenvalues'][:,0] - EF,'k--',label='GPAW')

    # for ii in high_sym_kpoints:
    #     plt.axvline(ii, color='gray', lw=1, ls='--')
    # plt.tick_params(direction='in')
    # plt.ylim(-2, 35)
    # plt.xlim(xlist.min(), xlist.max())
    # plt.ylabel('E - EF (eV)', fontsize=12)
    # plt.yticks(fontsize=12)
    # plt.xticks(high_sym_kpoints, labels, fontsize=12)
    # plt.show()



# N-N: 7  independent hoppings
## 2s-2s: [0]    2s-2p: [1]    2s-s*: [2]    2p-2s: [1]    2p-2p: [3, 4]    2p-s*: [5]    
## s*-2s: [2]    s*-2p: [5]    s*-s*: [6]    
# N-B: 10  independent hoppings
## 2s-2s: [0]    2s-2p: [1]    2s-s*: [2]    2p-2s: [3]    2p-2p: [4, 5]    2p-s*: [6]    
## s*-2s: [7]    s*-2p: [8]    s*-s*: [9]    
# B-N: 10  independent hoppings
## 2s-2s: [0]    2s-2p: [3]    2s-s*: [7]    2p-2s: [1]    2p-2p: [4, 5]    2p-s*: [8]    
## s*-2s: [2]    s*-2p: [6]    s*-s*: [9]    
# B-B: 7  independent hoppings
## 2s-2s: [0]    2s-2p: [1]    2s-s*: [2]    2p-2s: [1]    2p-2p: [3, 4]    2p-s*: [5]    
## s*-2s: [2]    s*-2p: [5]    s*-s*: [6]    
# N: {'2s': [0], '2p': [1], 's*': [2]}  independent onsite Es
## 2s: [0]    2p: [1]    s*: [2]    
# B: {'2s': [0], '2p': [1], 's*': [2]}  independent onsite Es
## 2s: [0]    2p: [1]    s*: [2]    
# initial rotate H or S func.
# N-N: 7  independent hoppings
## 2s-2s: [0]    2s-2p: [1]    2s-s*: [2]    2p-2s: [1]    2p-2p: [